$\newcommand{\vect}[1]{\boldsymbol{#1}}$

# 6. Modelling
## Introduction
- The application of network theory is really <u>multidisciplinary</u> and faces similar problems across the various topics.
- It is always important to be able to reproduce <mark><b>synthetically</b></mark> in a computer the system we are studying.
- This is the basis of the art of modelling, which allows various and important applications.


- Models
    - to predict the outcome of an experiment.
    - to understand the basic principles shaping the of a given phenomenon.
    - It is always validation with data from experiments that allows us to determine if the model hypothesis is right or not.

- The aims of this chapter
    - <u>to present the most used models in the fìeld of complex networks</u>,
    - to illustrate the basic principles on which they are based (sometimes inspired by similar situations in different fìelds),
    - to provide the code for them.

## Exponential growth, chains, and random graph

### Static models
#### One species: Fibonacci sequence
- The simplest assumption of a food web, <u>on a short time scale</u> where mutation does not happen is when we deal with one species and unlimited resources.
- This is a <u>highly unlikely assumption</u> both in ecology and in other fields, but it is an example that is important historically and in our opinion pleasant to describe.
<img src="table6.1.png" width=320>
<center><font size=-1>[counts the pairs of rabbits. The formula is 
$n(t) = n(t - 1) + n(t - 2)$]</font></center>


- As $t\rightarrow\infty$  we have that the ratio of two consecutive terms approaches a constant value that is $\lim_{t\rightarrow\infty}n(t) = \phi n(t - 1)$ (such a value of $\phi$ becomes more and more similar to the "golden ratio", $\phi^G =\frac{1+\sqrt{5}}{2}\simeq 1.61803\cdots$, a number particularly used by architects and artists to mimic the beauty of nature).
$$\frac{dn}{dt}=\frac{n(t+1)-n(t)}{1}\simeq(\phi -1)n(t)$$
whose solution is $n(t)=n(t_0)e^{(\phi -1)(t-t_0)}$.
- Ref
    - [황금비](http://terms.naver.com/entry.nhn?docId=3582431&cid=58714&categoryId=58714)
    - [피보나치 수열](http://terms.naver.com/entry.nhn?docId=3338362&cid=47324&categoryId=47324)

In [ ]:
# Compute the Fibonacchi sequence
def fibonacci(sequence_length):
    sequence = [0, 1]
    if sequence_length < 1:
        print("Fibonacci sequence only defined for length 1 or greater")
        return
    if 0 < sequence_length < 3:
        return sequence[:sequence_length]
    for i in range(2, sequence_length):
        sequence.append(sequence[i-1] + sequence[i-2])
    return sequence

fibonacci(2)

In [ ]:
def fib_recursive(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)
    
print([fib_recursive(i) for i in range(12)])

#### Two species: Lotka-Volterra equations
- The simplest way to describe a more realistic food web is to introduce at least one prey and one predator.
- In the simplest version with one prey and one predator z,we have
$$\begin{cases} \textbf{y'(t)}=\alpha\textbf{y(t)z(t)}-\beta\textbf{y(t)}\\ \textbf{z'(t)}=\gamma\textbf{z(t)}-\delta\textbf{z(t)y(t)} \end{cases}$$
- For the predator $y(t)$:
    - the first term $\alpha y(t)z(t)$ describes the growth based on predation on $z$. It is an exponential growth (the factor $y(t)$, where the resources are proportional to the prey $\alpha z(t)$;
    - the second item $\beta y(t)$ takes into account death or emigration.
- For the prey $z$:
    - the first term $\gamma z(t)$ is an exponential growth (assuming infinite and constant resources)
    - the second term $\delta z(t)y(t)$ accounts for predation from predators. Since not all the losses by $z$ contribute to the growth of $y$, this term is different from the first term in the predator equation.
    
- Ref
    - https://ko.wikipedia.org/wiki/로트카-볼테라_방정식
    - https://en.wikipedia.org/wiki/Lotka–Volterra_equations
    - http://blog.naver.com/dydrogud22/220483569583

- [SymPy](http://www.sympy.org/en/index.html) : a Python library for symbolic mathematics.
- <code>$ pip install sympy</code>

In [ ]:
# Solve the Lotka-Volterra differential equations

from sympy import *

#defining the variables and parameters
var('y z')
var('alfa beta gamma delta', positive=True)

#defining the equations
dy = alfa*z*y - beta*y
dz = gamma*z - delta*z*y

#solving the Lotka Volterra equations
(y0, z0), (y1, z1) = solve([dy, dz], (y, z))
A = Matrix((dy, dz))
print("A = ", A, "\n")

#computing the Jacobia
Jacobian = A.jacobian((y, z))
print("Jacobian =", Jacobian)
B = Jacobian.subs(y, y0).subs(z, z0)
C = Jacobian.subs(y, y1).subs(z, z1)
print("B = ", B)
print("C = ", C, "\n")

#stability of the fixed points
solutionB = B.eigenvals()
solutionC = C.eigenvals()
print("Solution B = ", solutionB)
print("Solution C = ", solutionC)

- Ref
    - https://ko.wikipedia.org/wiki/야코비_행렬
    - http://dexterstory.tistory.com/777
    - http://scipy-cookbook.readthedocs.io/items/LoktaVolterraTutorial.html

#### Many species, random competition
- When adding more and more species to the system, the set of relationships can be effectively described by means of a predation matrix $\vect{A}$ whose entries $a_{ij}$ have the following property
$$a_{ij}=\begin{cases} 1 & \text{if $j$ predates $i$}\\ 0 & \text{otherwise ($j$ doese not predate $i$)} \end{cases}$$
- That is $\vect{A}$ is the matrix of an oriented graph where the <u>out-degree gives the number of predators</u> (the arrows indicate the flow of nutrients). 

## Random graphs
- Random graphs (Erdös and Rényi,1959) represent the benchmark for any real network.
- In a random graph, once given the number of vertices and the probability $p$ to draw an edge between a couple of them, we obtain a graph where connections are assigned randomly.
$$P_k=\frac{(n-1)!}{(n-1-k)!k!}p^k(1-p)^{n-1-k}=\left(\matrix{n-1 \cr k}\right) p^k(1-p)^{n-1-k}$$

- That is, we find a binomial distribution for the various values of degree k; note that the distribution is automatically normalised since
$$\sum_{k=1,n-1}P_k = (p+(1-p))^{n-1}=1$$
- A huge amount of analytical work has been done with random graphs (Bollobás,1979; Bollobás,1985) and those analytical results constitute the <u>basic benchmark to evaluate the behaviour of any other graph</u>.
- Ref
    - https://en.wikipedia.org/wiki/Random_graph

In [ ]:
%matplotlib inline

In [ ]:
# Generating Random Networks (Erdös-Rényi)

import networkx as nx
import random

Number_of_nodes = 10
p = 0.4

G = nx.Graph()
for n in range(Number_of_nodes):
    G.add_node(n)
    
node_list = G.nodes()

for i1 in range(len(node_list)-1):
    for i2 in range(i1+1, len(node_list)):
        if random.random() < p:
            G.add_edge(node_list[i1], node_list[i2])

pos = nx.circular_layout(G)
nx.draw(G, pos, with_labels=True)

- By above approach allows us to reproduce a variety of different behaviours, but does not necessarily allow us to reproduce the real complexity of a graph, with correlation between vertices made evident by non-trivial values of assortativity and clustering.

In [ ]:
# Generating Random Networks (Erdös-Rényi)
# Modified by etc(2017/07/20)

import networkx as nx
import matplotlib.pyplot as plt
import random


Number_of_nodes = 10
p = 0.4

fig = plt.figure()
plt.ion()

G = nx.Graph()
for n in range(Number_of_nodes):
    G.add_node(n)
    
node_list = G.nodes()

for i1 in range(len(node_list)-1):
    for i2 in range(i1+1, len(node_list)):
        rn = random.random()
        if rn < p:
            pos = nx.circular_layout(G)
            plt.axis('off')
            nx.draw_networkx_nodes(G, pos, with_labels=True)
            G.add_edge(node_list[i1], node_list[i2])
            plt.pause(0.2)
            nx.draw_networkx_edges(G, pos, with_labels=True)
            plt.show()
            plt.clf()
            plt.close()
            plt.pause(0.2)

# nx.draw(G, pos, with_labels=True)

### Randomising a graph
- Real networks display a long-range level of correlation.
- Not only is the degree <u>distribution scale-free</u>, but also <u>higher-order correlations</u> are present. It is then useful to have a method of characterising how much these correlations count with respect to a null case.
- This means that we would like to measure the properties of a real graph against another one where correlations are not present.
- This can be done by realising a randomisation of the initial graph, not at the level of the degree (a random graph with the same number of vertices and edges of a real network is very likely not to have the same degree sequence), but at a higher level.


- A procedure to destroy correlation by preserving degree sequence can easily be outlined as follows (Maslov et al. , 2004).
- Extract one couple of edges (A, B) and (C, D) with four different vertices (A, B, C, D) and swap the extremes (end vertices) ofthe two edges (i.e. (A-B), (C-D) → (C-B) , (A-D)) as outlined in figure below.


<img src="Fig6.1.png" width=400>

In [ ]:
# Randomising graphs

number_of_swaps = 2

while number_of_swaps > 0:
    # pick at random a couple of edges they don’t share nodes
    edges_to_swap = random.sample(G.edges(), 2)
    e0 = edges_to_swap[0]
    e1 = edges_to_swap[1]
    
    if len(set([e0[0], e0[1], e1[0], e1[1]])) < 4:
        continue
    
    # check if the edge already exists and eventually add it
    if not G.has_edge(e0[0], e1[1]):
        G.add_edge(e0[0], e1[1])
    G.remove_edge(e0[0], e0[1])
    if not G.has_edge(e0[1], e1[0]):
        G.add_edge(e0[1], e1[0])
    G.remove_edge(e1[0], e1[1])
    number_of_swaps -= 1

pos = nx.circular_layout(G)
nx.draw(G, pos, with_labels=True)

## Configuration models
- The importance of random graphs is due to the fact that they represent an instance of being a graph when no particular correlation in the vertices is present.
- Build a a graph starting from the basic constituents, the vertices with their given edge stubs.
<img src="Fig6.2.png" width=400>
1. order the vertices from the one with the largest degree to that with the smallest;
2. assign to any of them a number of edge stubs given by the degree sequence;
3. write down an array whose entries are all t he stubs indicated by the label of the
vertex to which they belong;
4. scramble t he array exchanging the position of the entries;
5. take the entries in block for two and draw an edge between the vertices indicated.

In [ ]:
# Configuration model

degree_sequence = [6, 4, 3, 2, 1, 1, 1]

# this generate the list of uppercast chars as labels for the nodes
uppercase_char_list = [chr(i) for i in range(65, 91)]

degree_sequence.sort(reverse=True)
print("degree sequence:", degree_sequence)

stub_list = []

for deg in degree_sequence:
    label = uppercase_char_list.pop(0)
    for stub in range(deg):
        stub_list.append(label)
        
print("ordered stub labels", stub_list)

random.shuffle(stub_list)

print("shuffled stub labels", stub_list)

MG = nx.MultiGraph()

while stub_list != []:
# while stub_list:
    node1 = stub_list.pop(0)
    node2 = stub_list.pop(0)
    MG.add_edge(node1, node2)
    
print("graph edge list:", MG.edges())

pos = nx.circular_layout(MG)
nx.draw(MG, pos, with_labels=True)

graph edge list: [('B', 'A'), ('B', 'G'), ('B', 'C'), ('B', 'D'), ('F', 'C'), ('D', 'A'), ('E', 'C'), ('A', 'A'), ('A', 'A')]

## Gravity model
- Inspired by the Newtonian law of forces,one can imagine that the flow of trade between two countries might depend on the two “masses" of t he countries involved and might be inversely proportional (not necessarily with a square power) to their reciprocal distance.
$$T_{ij}=G(M^{\beta_1}_i M^{\beta_2}_j/D^{\beta_3}_{ij})$$
$T_{ij}$ indicates the trade between two countries $i,j$; the masses $M_{i,j}$ are characteristic of the countries; $i,j,D_{i,j}$ is their reciprocal distance (between the capitals), and $G$ is a constant.
-  There are various explanations for the meaning of the masses $M_{i,j}$ for the countries $i,j$. 
- The simplest choice is to use the GDP of countries as the quantity $M$ describing the trade $F$ between two nations. 
- The values of the exponents $\beta_{1,2,3}$ are determined from data by means of various methods of fitting.
- Ref
    - http://geoworld.pe.kr/geogrdic/html/9-021.htm

In [ ]:
# Gravity model
import scipy.optimize as optimization
import matplotlib.pyplot as plt
import numpy as np

# Generate artificial data straight line with a=O and b=l 
# plus some noise.
xdata = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])
ydata = np.array([0.1, 0.9, 2.2, 2.8, 3.9, 5.1])

sigma = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

x0 = np.array([0.0, 0.0, 0.0])

def func(x, a, b, c):
    return a + b*x + c*x*x

popt, pcov = optimization.curve_fit(func, xdata, ydata, x0, sigma)
print(popt, "\n", pcov)

plt.plot(xdata, ydata, 'b-', label='data')
plt.plot(xdata, func(xdata, *popt), 'r-', label='fit')

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

## Fitness model
- Along a similar line of reasoning to that for the gravity model, there is a whole class of network models that are based on some “mass" or quantity characteristic of the vertices. 
- This quantity indicated as “fitness" determines the property of the network, starting from the degree of every node.
- More particularly,the probability of drawing an edge is a function of the fitness of the vertices involved.


- Aassign a real value $X_i$ to every vertex $i$ of the graph. This value can be taken from data or extracted from a probability distribution $\rho(x)$.
- Determine a function $f(x_i,x_j)$ giving the probability $p_{i,j}$ with which the two vertices $(i,j)$ will be connected.
- According to the various possible choices for both the distribution $\rho(x)$ and the linking probability $p_{i,j}$ different classes of networks appear.
- The expected value $k(x)$ of the degree of a vertex whose fitness is $x$, we have
$$k(x) = N\int^\infty_0\rho(y)f(x, y)=NF(x)$$

- Invert it and in the limit of large $N$ (where
we can neglect finite corrections) we can write
$$P(k) = \rho\Big[F^{-1}(\frac{k}{n})\Big]\frac{d}{dk}F^{-1}(\frac{k}{n})$$
- If the fitnesses are exponentially distributed so that $x_i, x_j$  are extracted from a $\rho(x) = Ae^{-x}$, where $A=1$ is the normalisation constant and the linking probability $p_{i,j}$ is a threshold function $p_{i,j}=\Theta(x_i + x_j - z(N))$, we can obtain a scale-free network $P(k)\propto k^{-2}$.


- Ref
    - https://en.wikipedia.org/wiki/Fitness_model_(network_theory)

In [ ]:
# Fitness model
import matplotlib.pyplot as plt
import math

G = nx.Graph()

# this is our z(N)
ave_value = 1.0
N = 5000

def fitness_function():
    return random.expovariate(4.0 / ave_value)

def generate_function(x1, x2):
    if x1 + x2 - ave_value < 0.0:
        return 0
    else:
        return 1
    
for n in range(N):
    G.add_node(n, fitness=fitness_function())
    
node_list = G.nodes()

# generate the graph adding ad edge for each possible couple of nodes
for i1 in range(len(node_list)-1):
    for i2 in range(i1+1, len(node_list)):
        x1 = G.node[node_list[i1]]['fitness']
        x2 = G.node[node_list[i2]]['fitness']
        if generate_function(x1, x2) == 1:
            G.add_edge(node_list[i1], node_list[i2])
            
degree_sequence = sorted(nx.degree(G).values(), reverse=True)
plt.hist(degree_sequence, bins=15)

## Barabási-Albert model
- While the number of countries and their reciprocal distance can be considered to be (at least for the latter) fairly stationary in time, this is not the case for routers or web sites. In such cases a better suited statistical model is necessary.
- The most successful model of graph growth based on this totally different approach is the Barabási-Albert (BA) model (Barabási and Albert, 1999; Albert and Barabási, 2002).
- In this model the number of vertices varies continuously in time; in particular, time is discretised and at every time step a new vertex is added. A simple procedure for building a BA network is to start from a small initial graph and for every time step:
    1. introduce a new vertex in the system
    2. link the vertex to the initial graph by drawing $m$ new edges; the destination vertices are chosen with a probability proportional to their degree. This means that
    $$p=p(k_i)=\frac{k_i}{\sum_{j=1,n}k_j}.$$
    
    
- Note that, since at every time step only one vertex enters, we have for the order $n$, that is, the number of vertices and the size $m$ of the network, respectively
$$\begin{align*} n&=n_0+t, \\ m&=\frac{1}{2}\sum_{j=1,n}k_i=mt+m_0 \end{align*}$$


- The degree of any vertex can only increase, the variation of the degree of one node in one time step will be given by how many edges we add $(m)$ times the probability $p(k)$ to get an edge from the newcomer vertex just added. This means
$$\frac{\partial k_i}{\partial t}=m_o\Pi(k)=m\frac{k_i}{\sum_{j=1,n}k_j}=\frac{mk_i}{2mt+m_0}$$


- If we assume that the initial edges are $0$ (or in any case we study the behaviour for large $t$),we have
$$\frac{\partial k_i}{\partial t}=\frac{k_i}{2t}\rightarrow k_i(t)=m\Big(\frac{t}{t_i}\Big)^{1/2}$$
where $t_i$ is the time at which the vertex entered the systtem.


- From this first result (the degree grows with the square root power of time) we can derive the form of the degree distribution. The basic idea is that we can use time and degree interchangeably, this means that the probability $P(k_i < k)$ that a vertex has
a degree lower than $k$ is $P(k_i < k) = P(t_i > \frac{m^2_0t}{k^2}$. We also know that enter at a constant rate so that the time distribution is uniform in time,that is $P(t) = A$, where the constant value of $A$ can be determined by imposing $A =\int^n_0P(t) = An = 1$, so that $A = 1/n = 1/(n_0 +t)$. In this way, we write
$$P(t_i > \frac{m^2t}{k^2})=1-P(t_i\leq\frac{m^2t}{k^2})=1-\frac{m^2t}{k^2}\frac{1}{(n_0+t)}$$
$$P(k)=\frac{\partial P(k_i > k)}{\partial k}=\frac{2m^2t}{(n_0 + t)}\frac{1}{k^3}$$
- Therefore, we find that the degree distribution is a power law with a value of the exponent $\gamma = 3$.

- Ref
    - https://en.wikipedia.org/wiki/Barabási–Albert_model

In [ ]:
# Barabási-Albert model

import matplotlib.pyplot as plt

N0 = 6
p = 0.6
new_nodes = 1000

G = nx.gnp_random_graph(N0, p)

for eti in range(new_nodes):
    m = 3
    new_eti = "_"+str(eti)
    target_nodes = []
    while m != 0:
        part_sum = 0.0
        rn = random.random()
        for n in G.nodes():
            base = part_sum
            step = part_sum + G.degree(n)/(G.number_of_edges()*2.0)
            part_sum = part_sum + G.degree(n)/(G.number_of_edges()*2.0)
            if rn >= base and rn < step:
                if n in target_nodes:
                    break
                target_nodes.append(n)
                m = m - 1
                break
                
    for n_tar in target_nodes:
        G.add_edge(new_eti, n_tar)

degree_sequence = sorted(nx.degree(G).values(), reverse=True)

plt.hist(degree_sequence, bins=15)

## Reconstruction of networks
- Reconstructing a network when only limited information is available represents one of the major challenges in the field of complex systems,especially those concerning socio-economics:

#### Measuring likelihood of unknown configurations
- (1) making optimal use of the available information, and (2) drawing as unbiased as possible conclusions on the unknown portion of the system.
- MaxEnt (i.e. maximum entropy) principle
$$S=-\sum_G P(G)\ln{P(G)}$$


$$P(G)=\frac{e^{-H(G)}}{Z}=\frac{e^{-\sum_a\theta_a C_a}}{Z}$$
where $H(G)$ sums up the available information on the network $G$ and $Z=\sum_Ge^{-H(G)}$ normalises the probability coefficient to one. 





### Estimating the unknown parameters
- The MaxEnt principle leaves us with the problem of estimating the unknown parameters $\theta_i,\forall i$.
$$\frac{\partial\mathcal{L}(\overrightarrow{\theta})}{\partial \theta_i} = 0, \forall i$$ 
- The algorithm for estimating the parameters of the configuration model reads $k^*_i = x_ix_j / (1 +x_ix_j),\forall i$. Notice that, in this case, the probability that any two nodes $i$ and $j$ establish a connection is driven by the value of their degrees.

### From null models to reconstruction methods
- Enhanced Configuration Model (ECM)
$$p_{ij} = \frac{x_ix_jy_iy_j}{1+x_ix_jy_iy_j - y_iy_j}, \langle w_{ij}\rangle=\frac{p_{ij}}{1-y_iy_j},$$
whose unknown parameters can be estimated through solving the system
$$\begin{cases}
k_i = \sum_{j(\neq i)}p_{ij},\forall i \\
s_i = \sum_{j(\neq i)}\langle w_{ij}\rangle, \forall i
\end{cases}$$

- the ECM tells us that creating a connection and reinforcing an existing connection are processes obeying different rules,driving nodes towards integration or segregation

### The fitness model
- The unknown parameters defining $P(G)$ can be estimated through the maximisation of the likelihood function $\mathcal{L(\overrightarrow{\theta})}$. However, networks exist for which
the probability that any two nodes interact can be explicitly written in terms of non- structural quantities which are typical of the system under analysis.
- For the World Trade Web we have
$$p^{WTW}_{ij} = \frac{z\ {GDP}_i\ {GDP}_j}{1+z\ {GDP}_i\ {GDP}_j}$$
where the extra parameter $z$ can be tuned to reproduce the observed number of links, $L(G) = \sum_i\sum_{j(\neq i)}p^{WTW}_{ij}$.

### Bootstrapping a network
- It is noteworthy, this approach can be extended to deal with cases where the aforementioned information is known for <u>only some of the nodes</u>.
- a “reduced" likelihood estimation can be carried out,in order to determine $z$ (Cimini et al., 2015c; Musmeci et al.,2013):
$$\sum_{i\in I}k_i=\sum_{i\in I}\sum_{j(\neq i)}\tilde{p}_{ij}$$